# Data

In [ ]:
#!value --name exampledata
029A
980A
179A
456A
379A

In [ ]:
#!value --name data
463A
340A
129A
083A
341A

# Utilities

In [ ]:
void Print(object s) {
    Console.WriteLine(s);
}

public static string DebugPrint<T>(this IEnumerable<T> self) =>
        new StringBuilder("[")
            .AppendJoin(", ", self)
            .Append(']')
            .ToString();

### Imports

In [ ]:
using System;
using System.Collections;
using System.Text.RegularExpressions;
using System.Linq;
using System.Diagnostics;
using System.Numerics;

### Data Selector

In [ ]:
#!set --name fullData --value @value:data
#!set --name partialData --value @value:exampledata

var rawdata = fullData;
var data = rawdata.ReplaceLineEndings("\n");

# Part 1

```
+---+---+---+
| 7 | 8 | 9 |
+---+---+---+
| 4 | 5 | 6 |
+---+---+---+
| 1 | 2 | 3 |
+---+---+---+
    | 0 | A |
    +---+---+
```

```
    +---+---+
    | ^ | A |
+---+---+---+
| < | v | > |
+---+---+---+
```

In [ ]:
#!import coord.ipynb

// Function: Given a keypad and a sequence, output groups of sequences by length that would produce the target sequence.
// Represent keys as characters.

// A sequence is a set of pair-transitions. If we are on a given key, getting to another specified key always has a given best length.

char[][] NumericPad = [
    ['7', '8', '9'],
    ['4', '5', '6'],
    ['1', '2', '3'],
    ['X', '0', 'A']];

char[][] DirectionPad = [
    ['X', '^', 'A'],
    ['<', 'v', '>']];

Coord FindSymbol(char[][] map, char symbol) {
    for(int r = 0; r < map.Length; r++) {
        for(int c = 0; c < map[r].Length; c++) {
            if (map[r][c] == symbol) {
                return (r, c);
            }
        }
    }
    throw new System.InvalidOperationException("No Symbol?!");
}

var numerics = NumericPad.SelectMany(n => n).Where(n => n != 'X');
var directions = DirectionPad.SelectMany(d => d).Where(d => d != 'X');

In [ ]:
using Transition = (char from, char to);

Dictionary<Transition, List<string>> bestNumericPath = [];
foreach(char from in numerics) {
    foreach(char to in numerics) {
        Coord fromPos = FindSymbol(NumericPad, from);
        Coord toPos = FindSymbol(NumericPad, to);
        Coord bannedPos = FindSymbol(NumericPad, 'X');

        List<string> RecursivePathFrom(Coord pos) {
            // Print($"RPF {pos}");
            if (pos == toPos) return ["A"];
            Coord delta = Subtract(toPos, pos);
            List<string> results = [];
            switch(Math.Sign(delta.R)) {
                case 1:
                    Coord targetDown = Add(pos, (1, 0));
                    if (targetDown != bannedPos) {
                        results.AddRange(RecursivePathFrom(targetDown).Select(path => $"v{path}"));
                    }
                    break;
                case -1:
                    Coord targetUp = Add(pos, (-1, 0));
                    if (targetUp != bannedPos) {
                        results.AddRange(RecursivePathFrom(targetUp).Select(path => $"^{path}"));
                    }
                    break;
            }
            switch(Math.Sign(delta.C)) {
                case 1:
                    Coord targetRight = Add(pos, (0, 1));
                    if (targetRight != bannedPos) {
                        results.AddRange(RecursivePathFrom(targetRight).Select(path => $">{path}"));
                    }
                    break;
                case -1:
                    Coord targetLeft = Add(pos, (0, -1));
                    if (targetLeft != bannedPos) {
                        results.AddRange(RecursivePathFrom(targetLeft).Select(path => $"<{path}"));
                    }
                    break;
            }
            return results;
        }
        // Print($"Path from {from} -> {to}:  {fromPos} -> {toPos}");
        var result = RecursivePathFrom(fromPos);
        // display(result);
        bestNumericPath.Add((from, to), result);
    }
}

Dictionary<Transition, List<string>> bestDirectionalPath = [];
foreach(char from in directions) {
    foreach(char to in directions) {
        Coord fromPos = FindSymbol(DirectionPad, from);
        Coord toPos = FindSymbol(DirectionPad, to);
        Coord bannedPos = FindSymbol(DirectionPad, 'X');

        List<string> RecursivePathFrom(Coord pos) {
            // Print($"RPF {pos}");
            if (pos == toPos) return ["A"];
            Coord delta = Subtract(toPos, pos);
            List<string> results = [];
            switch(Math.Sign(delta.R)) {
                case 1:
                    Coord targetDown = Add(pos, (1, 0));
                    if (targetDown != bannedPos) {
                        results.AddRange(RecursivePathFrom(targetDown).Select(path => $"v{path}"));
                    }
                    break;
                case -1:
                    Coord targetUp = Add(pos, (-1, 0));
                    if (targetUp != bannedPos) {
                        results.AddRange(RecursivePathFrom(targetUp).Select(path => $"^{path}"));
                    }
                    break;
            }
            switch(Math.Sign(delta.C)) {
                case 1:
                    Coord targetRight = Add(pos, (0, 1));
                    if (targetRight != bannedPos) {
                        results.AddRange(RecursivePathFrom(targetRight).Select(path => $">{path}"));
                    }
                    break;
                case -1:
                    Coord targetLeft = Add(pos, (0, -1));
                    if (targetLeft != bannedPos) {
                        results.AddRange(RecursivePathFrom(targetLeft).Select(path => $"<{path}"));
                    }
                    break;
            }
            return results;
        }
        // Print($"Path from {from} -> {to}:  {fromPos} -> {toPos}");
        var result = RecursivePathFrom(fromPos);
        // display(result);
        bestDirectionalPath.Add((from, to), result);
    }
}

List<string> BestPathsForNumericSequence(string sequence) {
    IEnumerable<string> result = [""];
    for(int i = 0; i < sequence.Length - 1; i++) {
        var stepPaths = bestNumericPath[(sequence[i], sequence[i+1])];
        result = result.SelectMany(prefix => stepPaths.Select(suffix => prefix + suffix));
    }
    return result.ToList();
}
List<string> BestPathsForDirectionalSequence(string sequence) {
    IEnumerable<string> result = [""];
    for(int i = 0; i < sequence.Length - 1; i++) {
        var stepPaths = bestDirectionalPath[(sequence[i], sequence[i+1])];
        result = result.SelectMany(prefix => stepPaths.Select(suffix => prefix + suffix));
    }
    return result.ToList();
}

// BestPathsForNumericSequence("759");
// BestPathsForDirectionalSequence("A<");


In [ ]:
IEnumerable<string> LayerUp(IEnumerable<string> sequences) {
    var candidates = sequences.SelectMany(seq => BestPathsForDirectionalSequence("A" + seq));
    int bestLength = candidates.MinBy(seq => seq.Length).Length;
    return candidates.Where(seq => seq.Length == bestLength);
}

// foreach (string sequence in data.Split("\n")) {
//     var vacBot = BestPathsForNumericSequence("A" + sequence);
//     Print($"vacBot entries {vacBot.Count()}");
//     var radBot = LayerUp(vacBot);
//     Print($"radBot entries {radBot.Count()}");
//     var coldBot = LayerUp(radBot);
//     Print($"coldBot entries {coldBot.Count()}");
//     var me = LayerUp(coldBot);
//     Print($"me entries {me.Count()}");
// }

In [ ]:
Dictionary<Transition, List<string>> bestLevel1Path = [];
foreach(var (key, value) in bestNumericPath) {
    var shortestPaths = LayerUp(value);
    bestLevel1Path[key] = shortestPaths.ToList();
}
Dictionary<Transition, List<string>> bestLevel2Path = [];
foreach(var (key, value) in bestLevel1Path) {
    var shortestPaths = LayerUp(value);
    bestLevel2Path[key] = shortestPaths.ToList();
}
// Dictionary<Transition, List<string>> bestLevel3Path = [];
// foreach(var (key, value) in bestLevel2Path) {
//     var shortestPaths = LayerUp(value);
//     bestLevel3Path[key] = shortestPaths.ToList();
// }

int BestL2PathLengthForNumericSequence(string sequence) {
    int result = 0;
    for(int i = 0; i < sequence.Length - 1; i++) {
        var stepPaths = bestLevel2Path[(sequence[i], sequence[i+1])];
        result += stepPaths.First().Length;
    }
    return result;
}

// int BestL3PathLengthForNumericSequence(string sequence) {
//     int result = 0;
//     for(int i = 0; i < sequence.Length - 1; i++) {
//         var stepPaths = bestLevel3Path[(sequence[i], sequence[i+1])];
//         result += stepPaths.First().Length;
//     }
//     return result;
// }

In [ ]:
BigInteger answer = 0;
foreach (string sequence in data.Split("\n")) {
    var value = int.Parse(sequence.TrimEnd('A'));
    var result = BestL2PathLengthForNumericSequence("A" + sequence);
    // display($"{value} {result}");
    BigInteger complexity = value * result;
    answer += complexity;
}
answer

# Part 2


In [ ]:
// How much longer does a given transition get when we layer up?

Dictionary<Transition, List<string>> bestDirectionalMetaPath = [];
foreach(var (key, value) in bestDirectionalPath) {
    var shortestPaths = LayerUp(value);
    bestDirectionalMetaPath[key] = shortestPaths.ToList();
}
Dictionary<Transition, int> bestTransitionCost = [];
foreach(var (key, value) in bestDirectionalPath) {
    bestTransitionCost[key] = bestDirectionalMetaPath[key].First().Length;
}

// Shorter paths are better than longer paths.
// Paths with smaller transition-expansions are better than larger, length being

In [ ]:
int GetTransitionCost(string dSequence) {
    int cost = 0;
    for(int i = 0; i<dSequence.Length; i++) {
        char prev = i > 0 ? dSequence[i-1] : 'A';
        char next = dSequence[i];
        cost += bestTransitionCost[(prev, next)];
    }
    return cost;
}

In [ ]:
Dictionary<Transition, List<string>> prunedBestNumericPath = [];
foreach(var (key, value) in bestNumericPath) {
    int optimalTransitionCost = value.Select(GetTransitionCost).Min();
    prunedBestNumericPath[key] = value.Where(seq => GetTransitionCost(seq) == optimalTransitionCost).ToList();
}
Dictionary<Transition, List<string>> prunedBestDirectionalPath = [];
foreach(var (key, value) in bestDirectionalPath) {
    int optimalTransitionCost = value.Select(GetTransitionCost).Min();
    prunedBestDirectionalPath[key] = value.Where(seq => GetTransitionCost(seq) == optimalTransitionCost).ToList();
}

List<string> SmarterBestPathsForNumericSequence(string sequence) {
    IEnumerable<string> result = [""];
    for(int i = 0; i < sequence.Length - 1; i++) {
        var stepPaths = prunedBestNumericPath[(sequence[i], sequence[i+1])];
        result = result.SelectMany(prefix => stepPaths.Select(suffix => prefix + suffix));
    }
    return result.ToList();
}
List<string> SmarterBestPathsForDirectionalSequence(string sequence) {
    IEnumerable<string> result = [""];
    for(int i = 0; i < sequence.Length - 1; i++) {
        var stepPaths = prunedBestDirectionalPath[(sequence[i], sequence[i+1])];
        result = result.SelectMany(prefix => stepPaths.Select(suffix => prefix + suffix));
    }
    return result.ToList();
}
IEnumerable<string> SmarterLayerUp(IEnumerable<string> sequences) {
    var candidates = sequences.SelectMany(seq => SmarterBestPathsForDirectionalSequence("A" + seq));
    int bestLength = candidates.MinBy(seq => seq.Length).Length;
    return candidates.Where(seq => seq.Length == bestLength);
}

In [ ]:
// foreach (string sequence in data.Split("\n")) {
//     IEnumerable<string> layer = SmarterBestPathsForNumericSequence("A" + sequence);
//     for (int i = 0; i < 5; i++) {
//         layer = SmarterLayerUp(layer);
//         Print($"i:{i}  {layer.Count()}");
//     }
//     return layer;
// }

In [ ]:
// BigInteger answer = 0;
// foreach (string sequence in data.Split("\n")) {
//     var value = int.Parse(sequence.TrimEnd('A'));
//     var result = BestL2PathLengthForNumericSequence("A" + sequence);
//     // display($"{value} {result}");
//     BigInteger complexity = value * result;
//     answer += complexity;
// }
// answer

## What if we recompute the transition costs until they stabilize?

In [ ]:
// Prune directional paths based on both length and transition cost.
// (Done above)

// Recalculate transition cost based on the pruned paths.
Dictionary<Transition, int> recursiveTransitionCost = [];
Dictionary<Transition, List<string>> recursiveDirectionalMetaPath = [];
foreach(var (key, value) in prunedBestDirectionalPath) {
    var shortestPaths = LayerUp(value);
    recursiveDirectionalMetaPath[key] = shortestPaths.ToList();
}
foreach(var (key, value) in prunedBestDirectionalPath) {
    recursiveTransitionCost[key] = recursiveDirectionalMetaPath[key].First().Length;
}
Dictionary<Transition, List<string>> reprunedBestDirectionalPath = [];
int GetRecursiveTransitionCost(string dSequence) {
    int cost = 0;
    for(int i = 0; i<dSequence.Length; i++) {
        char prev = i > 0 ? dSequence[i-1] : 'A';
        char next = dSequence[i];
        cost += recursiveTransitionCost[(prev, next)];
    }
    return cost;
}
foreach(var (key, value) in prunedBestDirectionalPath) {
    int optimalTransitionCost = value
        .Select(GetRecursiveTransitionCost)
        .Min();
    reprunedBestDirectionalPath[key] = value
        .Where(
            seq => GetRecursiveTransitionCost(seq) == optimalTransitionCost
        ).ToList();
}

bool changed = false;
do {
    changed = false;
    foreach(var (key, value) in reprunedBestDirectionalPath) {
        var shortestPaths = LayerUp(value);
        recursiveDirectionalMetaPath[key] = shortestPaths.ToList();
    }
    foreach(var (key, value) in reprunedBestDirectionalPath) {
        recursiveTransitionCost[key] = recursiveDirectionalMetaPath[key].First().Length;
    }
    foreach(var (key, value) in reprunedBestDirectionalPath) {
        int optimalTransitionCost = value
            .Select(GetRecursiveTransitionCost)
            .Min();
        var newValue = value.Where(
            seq => GetRecursiveTransitionCost(seq) == optimalTransitionCost
        );
        if (newValue.Count() < value.Count) {
            reprunedBestDirectionalPath[key] = newValue.ToList();
        }
    }
    display(changed);
} while (changed);

